# Part 1 : Tracking moving object

In [3]:
import numpy as np
import cv2

# Paramètres globaux
N = 2000  # Augmenté pour un suivi plus précis
sigma_position = np.diag([20, 20])  # Réduit pour une prédiction plus stable en position
sigma_angle = 1  # Réduit pour une rotation plus stable
lambd = 6  # Paramètre de lissage pour la mise à jour des poids

# Positions et dimensions connues du centre
x, y, w, h = 301, 219, 41, 41  # Valeurs fournies

# Initialisation de la région d'intérêt (ROI) avec les valeurs connues
roi = (x, y, w, h)

# Chargement de la vidéo
cap = cv2.VideoCapture('video sequences/synthetic/escrime-4-3.avi')

# Lire la première image pour obtenir les dimensions
ret, frame = cap.read()
if not ret:
    print("Erreur : Impossible de lire la vidéo")
    cap.release()
    exit()

# Initialiser le VideoWriter pour enregistrer en MP4
output_file = "tracking_output_part2.mp4"
fourcc = cv2.VideoWriter_fourcc(*'mp4v')  # Codec pour MP4
fps = 30  # Fréquence d'images, ajustez si nécessaire
frame_size = (frame.shape[1], frame.shape[0])
out = cv2.VideoWriter(output_file, fourcc, fps, frame_size)

# Calcul de l'histogramme en HSV pour une meilleure robustesse aux variations de couleur
def calc_histogram(image, roi):
    x, y, w, h = roi
    roi_image = image[int(y):int(y+h), int(x):int(x+w)]
    hsv_roi = cv2.cvtColor(roi_image, cv2.COLOR_BGR2HSV)  # Convertir en HSV
    hist = cv2.calcHist([hsv_roi], [0, 1], None, [180, 256], [0, 180, 0, 256])  # Histogramme en HSV
    hist = cv2.normalize(hist, hist).flatten()
    return hist

roi_hist = calc_histogram(frame, roi)

# Initialisation des particules avec (x, y, theta)
particles = np.column_stack((
    np.random.uniform(x, x + w, N),  # Position x
    np.random.uniform(y, y + h, N),  # Position y
    np.full(N, 0)  # Angle theta initialisé à 0 pour une orientation initiale stable
))
weights = np.ones(N) / N  # Poids initiaux uniformes

# Fonction de prédiction pour la position et l'angle avec bruit réduit pour plus de stabilité
def predict(particles, sigma_position, sigma_angle):
    noise_position = np.random.multivariate_normal([0, 0], sigma_position, N)
    noise_angle = np.random.normal(0, sigma_angle, N)
    particles[:, 0:2] += noise_position
    particles[:, 2] += noise_angle
    particles[:, 2] %= 360
    return particles

# Calcul de l'histogramme en tenant compte de l'angle de rotation
def calc_histogram_with_orientation(image, particle, w, h):
    x, y, theta = particle
    M = cv2.getRotationMatrix2D((x,y), theta, 1.0)
    rotated_roi = cv2.warpAffine(image, M, (image.shape[1], image.shape[0]))
    roi = rotated_roi[int(y - h / 2):int(y + h / 2), int(x - w / 2):int(x + w / 2)]
    hsv_roi = cv2.cvtColor(roi, cv2.COLOR_BGR2HSV)
    hist = cv2.calcHist([hsv_roi], [0, 1], None, [180, 256], [0, 180, 0, 256])
    hist = cv2.normalize(hist, hist).flatten()
    return hist

# Mise à jour des poids en fonction de la similarité des histogrammes
def compute_weights(particles, frame, roi_hist, lambd):
    weights = np.zeros(N)
    for i, particle in enumerate(particles):
        patch_hist = calc_histogram_with_orientation(frame, particle, w, h)
        dist = cv2.compareHist(roi_hist, patch_hist, cv2.HISTCMP_BHATTACHARYYA)
        weights[i] = np.exp(-lambd * dist**2)
    weights /= np.sum(weights)
    return weights

# Rééchantillonnage des particules en fonction des poids
def resample(particles, weights):
    indices = np.random.choice(N, N, p=weights)
    return particles[indices]

# Filtre de lissage pour stabiliser l'angle
def smooth_angle(previous_angle, current_angle, alpha=0.7):
    diff = (current_angle - previous_angle + 180) % 360 - 180
    return previous_angle + alpha * diff

previous_angle = 0  # Initialiser l'angle précédent pour le lissage


# Boucle principale de suivi sans affichage en temps réel
while True:
    ret, frame = cap.read()
    if not ret:
        break

    # Prédiction des particules (x, y, theta)
    particles = predict(particles, sigma_position, sigma_angle)
    
    # Correction et mise à jour des poids
    weights = compute_weights(particles, frame, roi_hist, lambd)
    
    # Rééchantillonnage
    particles = resample(particles, weights)

    # Estimation de la position et de l'angle en prenant la moyenne pondérée
    estimated_position = np.average(particles[:, :2], axis=0, weights=weights)
    raw_estimated_angle = np.average(particles[:, 2], weights=weights)
    estimated_angle = smooth_angle(previous_angle, raw_estimated_angle)
    previous_angle = estimated_angle  # Mise à jour de l'angle précédent

    # Dessiner le rectangle orienté
    oriented_rect = ((estimated_position[0], estimated_position[1]), (w, h), estimated_angle)
    box = cv2.boxPoints(oriented_rect)
    box = np.intp(box)
    cv2.drawContours(frame, [box], 0, (0, 255, 0), 2)

    # Écrire chaque particule en jaune
    for particle in particles:
        cv2.circle(frame, (int(particle[0]), int(particle[1])), 1, (0, 255, 255), -1)

    # Écrire la frame dans le fichier de sortie
    out.write(frame)

# Libérer les ressources 
cap.release()
out.release()
cv2.destroyAllWindows()

print(f"Vidéo enregistrée sous le nom : {output_file}")

Vidéo enregistrée sous le nom : tracking_output_part2.mp4


# Part 2 - Tracking complex objects in videos

In [26]:
import cv2
import numpy as np
import matplotlib.pyplot as plt

# Paramètres de base
video_path = 'video sequences/synthetic/escrime-4-3.avi'  # Chemin vers la vidéo
output_file = "tracking_output.mp4"  # Chemin pour enregistrer la sortie
N = 200  # Nombre de particules par objet
sigma = np.diag([40, 40])  # Bruit gaussien pour la prédiction
lambda_likelihood = 10  # Paramètre lambda pour la vraisemblance
hist_bins = 256  # Nombre de bins pour l'histogramme de couleurs

# Coordonnées des différentes parties
regions = {
    "center": (301, 219, 41, 41),  # (x, y, w, h)
    "arm1": (271, 234, 30, 12),
    "arm2": (316, 188, 12, 30),
    "arm3": (342, 235, 30, 12),
    "arm4": (315, 261, 12, 30)
}

# Charger la vidéo
cap = cv2.VideoCapture(video_path)
ret, frame = cap.read()
if not ret:
    print("Erreur : Impossible de lire la vidéo")
    cap.release()
    exit()

# Initialiser le VideoWriter pour enregistrer en MP4
fourcc = cv2.VideoWriter_fourcc(*'mp4v')  # Codec pour MP4
fps = 30  # Fréquence d'images
frame_size = (frame.shape[1], frame.shape[0])
out = cv2.VideoWriter(output_file, fourcc, fps, frame_size)

# Fonction pour calculer l'histogramme d'une région
def calc_histogram(image, roi):
    x, y, w, h = roi
    roi_image = image[int(y):int(y+h), int(x):int(x+w)]
    hist = cv2.calcHist([roi_image], [0], None, [hist_bins], [0, 256])
    hist = cv2.normalize(hist, hist).flatten()
    return hist

# Calcul des histogrammes pour chaque région
roi_hists = {key: calc_histogram(frame, roi) for key, roi in regions.items()}

# Initialiser les particules et les poids pour chaque région
particles_list = []
weights_list = []
colors = [(0, 255, 255), (0, 255, 0), (255, 0, 0), (255, 0, 255), (255, 255, 0)]  # Couleurs pour chaque objet

for key, (x, y, w, h) in regions.items():
    particles = np.column_stack((
        np.random.uniform(x, x + w, N),
        np.random.uniform(y, y + h, N)
    ))
    weights = np.ones(N) / N  # Poids initiaux uniformes
    particles_list.append(particles)
    weights_list.append(weights)

# Fonction de prédiction pour chaque objet
def predict(particles, sigma):
    noise = np.random.multivariate_normal([0, 0], sigma, N)
    particles += noise
    return particles

# Calcul des poids des particules pour chaque objet
def compute_weights(particles, frame, roi_hist, w, h, lambd=lambda_likelihood):
    weights = np.zeros(N)
    for i, particle in enumerate(particles):
        px, py = particle
        patch_hist = calc_histogram(frame, (px - w / 2, py - h / 2, w, h))
        dist = cv2.compareHist(roi_hist, patch_hist, cv2.HISTCMP_BHATTACHARYYA)
        weights[i] = np.exp(-lambd * dist**2)
    weights /= np.sum(weights)
    return weights

# Fonction de rééchantillonnage des particules
def resample(particles, weights):
    N = len(particles)
    indices = np.arange(N)
    cumulative_sum = np.cumsum(weights)
    cumulative_sum[-1] = 1.0
    u0 = np.random.uniform(0, 1 / N)
    positions = (u0 + np.arange(N) / N)
    new_indices = np.searchsorted(cumulative_sum, positions)
    resampled_particles = particles[new_indices]
    return resampled_particles

# Boucle principale
frame_skip = 1  # Sauter une frame sur deux
while True:
    for _ in range(frame_skip):  # Sauter des frames si nécessaire
        ret, frame = cap.read()
        if not ret:
            print("Erreur de lecture de la frame, fin de la vidéo.")
            cap.release()
            break

    if frame is None:
        break

    # Prédiction : mise à jour des particules avec du bruit
    for i in range(len(particles_list)):
        particles_list[i] = predict(particles_list[i], sigma)

    # Calcul des poids et rééchantillonnage pour chaque objet
    for i, (key, (x, y, w, h)) in enumerate(regions.items()):
        weights_list[i] = compute_weights(particles_list[i], frame, roi_hists[key], w, h)
        particles_list[i] = resample(particles_list[i], weights_list[i])

    # Calculer la position moyenne de chaque objet après rééchantillonnage
    estimated_positions = []
    for i in range(len(particles_list)):
        estimated_position = np.average(particles_list[i], axis=0, weights=weights_list[i])
        estimated_positions.append(estimated_position)

    # Dessiner les rectangles autour de chaque position estimée
    for i, est_pos in enumerate(estimated_positions):
        x, y, w, h = list(regions.values())[i]
        cv2.rectangle(frame,
                      (int(est_pos[0] - w / 2), int(est_pos[1] - h / 2)),
                      (int(est_pos[0] + w / 2), int(est_pos[1] + h / 2)),
                      colors[i], 2)

    # Afficher les particules pour chaque objet avec une couleur différente
    for i in range(len(particles_list)):
        for particle in particles_list[i]:
            cv2.circle(frame, (int(particle[0]), int(particle[1])), 2, colors[i], -1)

    # Écrire la frame dans le fichier de sortie
    out.write(frame)

    # Afficher l'image avec les objets suivis
    cv2.imshow("Tracking", frame)

    # Quitter en appuyant sur 'q'
    if cv2.waitKey(30) & 0xFF == ord('q'):
        break

# Libérer les ressources
cap.release()
out.release()  # Fermer le fichier vidéo de sortie
cv2.destroyAllWindows()

print(f"Vidéo enregistrée sous le nom : {output_file}")

Erreur de lecture de la frame, fin de la vidéo.
Vidéo enregistrée sous le nom : tracking_output.mp4
